#### Dockerfile creation
we first create a Dockerfile based on the documentation here https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-jl-admin-guide-custom-images.html#studio-updated-jl-custom-images-dockerfile-templates

```
FROM public.ecr.aws/sagemaker/sagemaker-distribution:latest-cpu
ARG NB_USER="sagemaker-user"
ARG NB_UID=1000
ARG NB_GID=100

ENV MAMBA_USER=$NB_USER

USER root

RUN apt-get update
RUN micromamba install sagemaker-inference --freeze-installed --yes --channel conda-forge --name base

USER $MAMBA_USER

ENTRYPOINT ["jupyter-lab"]
CMD ["--ServerApp.ip=0.0.0.0", "--ServerApp.port=8888", "--ServerApp.allow_origin=*", "--ServerApp.token=''", "--ServerApp.base_url=/jupyterlab/default"]
```

In [2]:
# we create the ECR repository
!aws ecr create-repository --repository-name smstudio-custom --region us-east-1


An error occurred (AccessDeniedException) when calling the CreateRepository operation: User: arn:aws:sts::794038231401:assumed-role/SageMaker-ExecutionRole-20250401T103257/SageMaker is not authorized to perform: ecr:CreateRepository on resource: arn:aws:ecr:us-east-1:794038231401:repository/smstudio-custom because no identity-based policy allows the ecr:CreateRepository action


In [3]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 794038231401.dkr.ecr.us-east-1.amazonaws.com 

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
!pip install sagemaker-studio-image-build

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'sagemaker-studio-image-build' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sagemaker-studio-image-build'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for sagemaker-studio-image-build: filename=sagemaker_studio_image_build-0.6.0-py3-none-any.whl size=13558 sha256=51873059472ad44a7677e081d40579e6ebe6fcea1d2a0b6740b2f3601e833dba
  Stored in directory: /home/ec2-user/.cache/pip/wheels/69/7b/d1/1318b1530ee5322c9be00f206badea11b5148626ef58c0e0dc
Successfully built sagemaker-studio-image-build


In [7]:
!sm-docker build -t smstudio-custom:my-jupyterlab-image .

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Created ECR repository sagemaker-studio
....................[Container] 2025/06/19 00:55:32.671091 Running on CodeBuild On-demand

[Container] 2025/06/19 00:55:32.671139 Waiting for agent ping
[Container] 2025/06/19 00:55:32.772127 Waiting for DOWNLOAD_SOURCE
[Container] 2025/06/19 00:55:33.097254 Phase is DOWNLOAD_SOURCE
[Container] 2025/06/19 00:55:33.098501 CODEBUILD_SRC_DIR=/codebuild/output/src3964233945/src
[Container] 2025/06/19 00:55:33.099176 YAML location is /codebuild/output/src3964233945/src/buildsp

In [ ]:
# 794038231401.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio:latest

# Create an AppImageConfig:

```
{
  "AppImageConfigName": "my-custom-image-config",
  "KernelGatewayImageConfig": {
    "KernelSpecs": [
      {
        "Name": "python3",
        "DisplayName": "Python 3"
      }
    ]
  },
  "FileSystemConfig": {
    "MountPath": "/home/sagemaker-user",
    "DefaultUid": 1000,
    "DefaultGid": 100
  }
}
```

In [9]:
!aws sagemaker create-app-image-config --cli-input-json file://app-image-config-input.json --region us-east-1

{
    "AppImageConfigArn": "arn:aws:sagemaker:us-east-1:794038231401:app-image-config/my-custom-image-config"
}


In [ ]:
# arn:aws:sagemaker:us-east-1:794038231401:app-image-config/my-custom-image-config

In [10]:
!aws sagemaker create-image \
  --image-name my-jupyterlab-image \
  --role-arn arn:aws:iam::794038231401:role/service-role/SageMaker-ExecutionRole-20250401T103257 \
  --region us-east-1

{
    "ImageArn": "arn:aws:sagemaker:us-east-1:794038231401:image/my-jupyterlab-image"
}


In [ ]:
# arn:aws:sagemaker:us-east-1:794038231401:image/my-jupyterlab-image

In [12]:
!aws sagemaker create-image-version \
  --image-name my-jupyterlab-image \
  --base-image 794038231401.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio:latest \
  --region us-east-1

{
    "ImageVersionArn": "arn:aws:sagemaker:us-east-1:794038231401:image-version/my-jupyterlab-image/1"
}


In [ ]:
# arn:aws:sagemaker:us-east-1:794038231401:image-version/my-jupyterlab-image/1

In [14]:
!aws sagemaker update-domain \
  --domain-id d-oc09trpafuvq \
  --default-space-settings file://default-space-settings.json \
  --region us-east-1

{
    "DomainArn": "arn:aws:sagemaker:us-east-1:794038231401:domain/d-oc09trpafuvq"
}


In [15]:
!aws sagemaker describe-domain --domain-id d-oc09trpafuvq

{
    "DomainArn": "arn:aws:sagemaker:us-east-1:794038231401:domain/d-oc09trpafuvq",
    "DomainId": "d-oc09trpafuvq",
    "DomainName": "QuickSetupDomain-20250509T182763",
    "HomeEfsFileSystemId": "fs-06f60e12721575183",
    "Status": "InService",
    "CreationTime": 1746836843.076,
    "LastModifiedTime": 1750296779.43,
    "AuthMode": "IAM",
    "DefaultUserSettings": {
        "ExecutionRole": "arn:aws:iam::794038231401:role/service-role/AmazonSageMaker-ExecutionRole-20250509T182764",
        "SharingSettings": {
            "NotebookOutputOption": "Allowed",
            "S3OutputPath": "s3://sagemaker-studio-794038231401-bdnwp0hqy1s/sharing"
        },
        "JupyterServerAppSettings": {
            "DefaultResourceSpec": {
                "SageMakerImageArn": "arn:aws:sagemaker:us-east-1:081325390199:image/jupyter-server-3",
                "InstanceType": "system"
            }
        },
        "CanvasAppSettings": {
            "TimeSeriesForecastingSettings": {
         